In [1]:
from access_data import *
from preparing_data import *
from evaluating_data import *
# from feature_engineering import *

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#getting all data
datasets = get_df_map()
thresholdMap = {}

for key, df in datasets.items():
    #splitting data
    split_map_all = split_dataset(df, validate_set=True)
    X_train_all = split_map_all['X_train']
    y_train_all = split_map_all['y_train']
    X_validate_all = split_map_all['X_validate']
    y_validate_all = split_map_all['y_validate']
    X_test_all = split_map_all['X_test']
    y_test_all = split_map_all['y_test']

    #encoding label    
    integer_encoded_train = y_train_all.replace("NONE", 0).replace(key, 1)
    integer_encoded_validate = y_validate_all.replace("NONE", 0).replace(key, 1)
    integer_encoded_test = y_test_all.replace("NONE", 0).replace(key, 1)  
    
    tfidf_vectorizer =  TfidfVectorizer(stop_words='english', max_df=0.15, min_df=5)
    X_train = tfidf_vectorizer.fit_transform(X_train_all)
    X_validate = tfidf_vectorizer.transform(X_validate_all)
    X_test = tfidf_vectorizer.transform(X_test_all)

    logReg = LogisticRegression(C=5)
    
    logReg.fit(X_train, integer_encoded_train)
    # predict probabilities
    probs = logReg.predict_proba(X_validate)
    predictions = convert_probability_list(probs)     
    predict_test_baseline = logReg.predict(X_test)
    
    # prediction on test set
    probs_test = logReg.predict_proba(X_test)
    predictions_test = convert_probability_list(probs_test)  
        
    thresholds = {}
    for i in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 
              0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:        
        predictions_binary = convert_probability_trueFalse(predictions, i)
        binary = convert_probability_trueFalse(predictions_test, 0.5)
        
        acc = accuracy_score(integer_encoded_test,binary)
        f1_score_micro = f1_score(integer_encoded_validate, predictions_binary, average='micro')        
        print('############## F1 score for label {} with thrashold {}: {} ###############'.format(key, i, f1_score_micro))
        thresholds[i] = f1_score_micro
       
    thresh = 0
    best_f1 = 0
    for key, value in thresholds.items():        
        if best_f1 <= value:
            best_f1 = value
            thresh = key

    
    print('###################### {} ####################'.format(key))
    print('--------------------------baseline------------------------') 
    print(y_test_all.value_counts())
    report = classification_report(integer_encoded_test, predict_test_baseline)
    print(report)
    print('--------------------adjusted threshold {}-------------------'.format(thresh))
    prediction_bin_test = convert_probability_trueFalse(predictions_test, thresh)
    report_test = classification_report(integer_encoded_test, prediction_bin_test)
    print(report_test)

labelSize: 40806
randomSize: 69818
Needed: 95214
labelSize: 34295
randomSize: 76329
Needed: 80022
labelSize: 6074
randomSize: 104550
Needed: 14173
labelSize: 5934
randomSize: 104690
Needed: 13846
labelSize: 6632
randomSize: 103992
Needed: 15475
labelSize: 9296
randomSize: 101328
Needed: 21691
labelSize: 6226
randomSize: 104398
Needed: 14528
labelSize: 482
randomSize: 110142
Needed: 1125
labelSize: 879
randomSize: 109745
Needed: 2051
############## F1 score for label ENTERTAINMENT with thrashold 0.05: 0.6542009223982355 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.1: 0.7524062562662923 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.15: 0.8075997593743732 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.2: 0.846551032684981 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.25: 0.8709645077200721 ###############
############## F1 score for label ENTERTAINMEN

In [7]:
print(type(y_test_all))

<class 'pandas.core.series.Series'>
